In [119]:
def readHeader(pos,msg):
  print("H Enter@",pos)
  version = int(msg[pos:pos+3],base=2)
  typeId = int(msg[pos+3:pos+6],base = 2)
  print("H Leave@p",pos,"V:",version,"typeId:",typeId)
  return (pos+6,version,typeId)

def readLiteral(pos,msg):
  print("L@",pos)
  resstring =""
  finished =False
  while not finished:
    tmp = msg[pos:pos+5]
    resstring += tmp[1:]
    if tmp[0] == '0':
      finished = True
    pos += 5
  print("L Leave:",pos," => ",int(resstring,base=2))
  return pos,int(resstring,base=2)

def readOperator(pos,msg):
  print("O@",pos,msg[pos])
  arr = []
  if msg[pos] == '0':
    sizebits = 15
    sizesubpackets = int(msg[pos+1:pos+1+sizebits],base=2)
    pos = pos +1+sizebits
    print("O:",pos,"size:",sizebits,"packlen:",sizesubpackets)
    endOfPacket = pos + sizesubpackets
    
    while pos < endOfPacket:
      print("O!@",pos,arr)
      pos,arr,_ = readPackage(pos,msg,arr)
      print("O!!@",pos,arr)
  else:
    numPacket = int(msg[pos+1:pos+1+11],base=2)
    print("O: numPackets",numPacket)
    pos = pos + 12
    subpackread = 0
    while subpackread < numPacket:
      pos,arr, numread = readPackage(pos,msg,arr)
      subpackread += numread
      print("O@",pos," ",subpackread," of ",numPacket)
    

  
  return pos,arr

def readPackage(pos,msg,arr):
  # read header
  print("P@",pos)
  numread = 0
  pos,version, typeID = readHeader(pos,msg)
  print("P@ after header:",pos)
  if (typeID == 4):
    pos, literal = readLiteral(pos,msg)
    arr.append([version,typeID,literal])
    numread = 1
  else:
    pos, oplist = readOperator(pos,msg)
    print("P!!@",pos,oplist,len(oplist))
    tmp = [[version,typeID]]
    tmp.extend(oplist)
    print("TMP",tmp)
    arr.append(tmp)
    numread = len(oplist) +1
  return pos, arr, numread
  #TODO 
  #END of reading

In [183]:
# lit1 = "38006F45291200"
# lit1 = "D2FE28"
#lit1 = "EE00D40C823060"
# lit1 = "8A004A801A8002F478"
lit1="620080001611562C8802118E34"
# lit1 = "C0015000016115A2E0802F182340"
# lit1 = "A0016C880162017C3686B18A3D4780"
# lit1 = "E20D41802B2984BD00540010F82D09E35880350D61A41D3004E5611E585F40159ED7AD7C90CF6BD6BE49C802DEB00525272CC1927752698693DA7C70029C0081002140096028C5400F6023C9C00D601ED88070070030005C2201448400E400F40400C400A50801E20004C1000809D14700B67676EE661137ADC64FF2BBAD745B3F2D69026335E92A0053533D78932A9DFE23AC7858C028920A973785338832CFA200F47C81D2BBBC7F9A9E1802FE00ACBA44F4D1E775DDC19C8054D93B7E72DBE7006AA200C41A8510980010D8731720CB80132918319804738AB3A8D3E773C4A4015A498E680292B1852E753E2B29D97F0DE6008CB3D4D031802D2853400D24DEAE0137AB8210051D24EB600844B95C56781B3004F002B99D8F635379EDE273AF26972D4A5610BA51004C12D1E25D802F32313239377B37100105343327E8031802B801AA00021D07231C2F10076184668693AC6600BCD83E8025231D752E5ADE311008A4EA092754596C6789727F069F99A4645008247D2579388DCF53558AE4B76B257200AAB80107947E94789FE76E36402868803F0D62743F00043A1646288800084C3F8971308032996A2BD8023292DF8BE467BB3790047F2572EF004A699E6164C013A007C62848DE91CC6DB459B6B40087E530AB31EE633BD23180393CBF36333038E011CBCE73C6FB098F4956112C98864EA1C2801D2D0F319802D60088002190620E479100622E4358952D84510074C0188CF0923410021F1CE1146E3006E3FC578EE600A4B6C4B002449C97E92449C97E92459796EB4FF874400A9A16100A26CEA6D0E5E5EC8841C9B8FE37109C99818023A00A4FD8BA531586BB8B1DC9AE080293B6972B7FA444285CC00AE492BC910C1697B5BDD8425409700562F471201186C0120004322B42489A200D4138A71AA796D00374978FE07B2314E99BFB6E909678A0"

In [184]:
litmsg = "0"+format(int(lit1,16),'b')

In [186]:
print(litmsg)

01100010000000001000000000000000000101100001000101010110001011001000100000000010000100011000111000110100


In [185]:
_, out,_ = readPackage(0,litmsg,[])

P@ 0
H Enter@ 0
H Leave@p 0 V: 3 typeId: 0
P@ after header: 6
O@ 6 1
O: numPackets 2
P@ 18
H Enter@ 18
H Leave@p 18 V: 0 typeId: 0
P@ after header: 24
O@ 24 0
O: 40 size: 15 packlen: 22
O!@ 40 []
P@ 40
H Enter@ 40
H Leave@p 40 V: 0 typeId: 4
P@ after header: 46
L@ 46
L Leave: 51  =>  10
O!!@ 51 [[0, 4, 10]]
O!@ 51 [[0, 4, 10]]
P@ 51
H Enter@ 51
H Leave@p 51 V: 5 typeId: 4
P@ after header: 57
L@ 57
L Leave: 62  =>  11
O!!@ 62 [[0, 4, 10], [5, 4, 11]]
P!!@ 62 [[0, 4, 10], [5, 4, 11]] 2
TMP [[0, 0], [0, 4, 10], [5, 4, 11]]
O@ 62   3  of  2
P!!@ 62 [[[0, 0], [0, 4, 10], [5, 4, 11]]] 1
TMP [[3, 0], [[0, 0], [0, 4, 10], [5, 4, 11]]]


In [181]:
print(out)

[[[7, 0], [[0, 1], [5, 4, 1161], [[3, 6], [[5, 0], [7, 4, 2], [6, 4, 4], [7, 4, 13]]]], [[3, 0], [2, 4, 6], [5, 4, 6], [4, 4, 3]], [[5, 1], [3, 4, 5250146], [[7, 6], [5, 4, 246310804], [1, 4, 246310804]]], [[7, 1], [[6, 5], [4, 4, 4401420], [4, 4, 55853]], [1, 4, 6]], [4, 4, 3759], [[4, 3], [[2, 3], [[0, 0], [[0, 2], [[1, 3], [[1, 2], [[7, 3], [[2, 3], [[5, 3], [[6, 1], [[0, 3], [[4, 0], [[2, 1], [[2, 0], [[6, 2], [[0, 0], [[4, 2], [[4, 1], [[2, 0], [[0, 2], [0, 4, 210]]]]]]]]]]]]]]]]]]]]], [[4, 3], [5, 4, 221698353], [0, 4, 12235055], [7, 4, 5985701], [5, 4, 993960], [0, 4, 562644]], [[5, 2], [2, 4, 2527], [0, 4, 53], [2, 4, 3749816546]], [[6, 1], [0, 4, 33], [2, 4, 118], [7, 4, 5], [1, 4, 200], [1, 4, 111]], [[5, 0], [6, 4, 15], [4, 4, 1], [6, 4, 1534196055]], [[4, 1], [[7, 7], [5, 4, 121], [0, 4, 3640]]], [7, 4, 14342924], [[7, 1], [5, 4, 57619135], [[4, 7], [[5, 0], [0, 4, 3], [2, 4, 5], [0, 4, 4]], [[6, 0], [5, 4, 7], [1, 4, 11], [4, 4, 3]]]], [[1, 3], [1, 4, 16652]], [[3, 1], [3,

In [76]:
a = [2,3]

In [77]:
print(a)

[2, 3]


In [142]:
def sumVersion(inp):
    result = 0
    if isinstance(inp[0],list):
        for i in inp:
            result += sumVersion(i)
    else:
        result = inp[0]
    return result

In [182]:
print(sumVersion(out))

329


In [187]:
h2b = {"0": "0000", "1": "0001", "2":"0010","3":"0011","4":"0100","5":"0101", "6":"0110","7":"0111","8":"1000","9": "1001","A":"1010","B":"1011","C":"1100","D":"1101","E": "1110","F":"1111"}

def get_n_bits(n,bits):
  r = bits[:n]
  bits = bits[n:]
  return r,bits

def b2d(b):
  weight = 1
  v = 0
  for i in reversed(range(len(b))):
    if b[i] == "1":
      v += weight
    weight = weight * 2
  return v

def process_literal(bits):
  bin_string = ""
  read_another = 1
  while read_another:
    cont,bits = get_n_bits(1,bits)
    b,bits = get_n_bits(4,bits)
    bin_string += b
    if cont == "0":
      read_another = 0
  v = b2d(bin_string)
  return v,bits 

def process_packet(bits):
  
  version,bits = get_n_bits(3,bits)
  ptype,bits = get_n_bits(3,bits)
  
  global version_sum
  version_sum += b2d(version)
  
  if ptype == "100":
    val,bits = process_literal(bits)
    print("literal value is:", val)
    return val, bits
  else:
    #operator packet
    length_type,bits = get_n_bits(1,bits)
    subp_vals = []
    if length_type == "0":
      print("operator packet, length type 0")
      len_string,bits = get_n_bits(15,bits)
      total_length_of_subpackets = b2d(len_string)
      subp,bits = get_n_bits(total_length_of_subpackets,bits)
      while subp:
        val, subp = process_packet(subp)
        subp_vals.append(val)      
    else:
      print("operator packet, length type 1")
      num_string,bits = get_n_bits(11,bits)
      num_of_subpackets = b2d(num_string)
      for i in range(num_of_subpackets):
        val, bits = process_packet(bits)
        subp_vals.append(val)
    
    if ptype == "000":
      v = sum(subp_vals)
    elif ptype == "001":
      v = 1
      for i in subp_vals:
        v = v * i
    elif ptype == "010":
      v = min(subp_vals)
    elif ptype == "011":
      v = max(subp_vals)
    elif ptype == "101":
      if subp_vals[0] > subp_vals[1]:
        v = 1
      else:
        v = 0
    elif ptype == "110":
      if subp_vals[0] < subp_vals[1]:
        v = 1
      else:
        v = 0
    elif ptype == "111":
      if subp_vals[0] == subp_vals[1]:
        v = 1
      else:
        v = 0
  return v, bits

#f = open("input16.txt","r")
#hex_string = f.readline().strip()
hex_string = "E20D41802B2984BD00540010F82D09E35880350D61A41D3004E5611E585F40159ED7AD7C90CF6BD6BE49C802DEB00525272CC1927752698693DA7C70029C0081002140096028C5400F6023C9C00D601ED88070070030005C2201448400E400F40400C400A50801E20004C1000809D14700B67676EE661137ADC64FF2BBAD745B3F2D69026335E92A0053533D78932A9DFE23AC7858C028920A973785338832CFA200F47C81D2BBBC7F9A9E1802FE00ACBA44F4D1E775DDC19C8054D93B7E72DBE7006AA200C41A8510980010D8731720CB80132918319804738AB3A8D3E773C4A4015A498E680292B1852E753E2B29D97F0DE6008CB3D4D031802D2853400D24DEAE0137AB8210051D24EB600844B95C56781B3004F002B99D8F635379EDE273AF26972D4A5610BA51004C12D1E25D802F32313239377B37100105343327E8031802B801AA00021D07231C2F10076184668693AC6600BCD83E8025231D752E5ADE311008A4EA092754596C6789727F069F99A4645008247D2579388DCF53558AE4B76B257200AAB80107947E94789FE76E36402868803F0D62743F00043A1646288800084C3F8971308032996A2BD8023292DF8BE467BB3790047F2572EF004A699E6164C013A007C62848DE91CC6DB459B6B40087E530AB31EE633BD23180393CBF36333038E011CBCE73C6FB098F4956112C98864EA1C2801D2D0F319802D60088002190620E479100622E4358952D84510074C0188CF0923410021F1CE1146E3006E3FC578EE600A4B6C4B002449C97E92449C97E92459796EB4FF874400A9A16100A26CEA6D0E5E5EC8841C9B8FE37109C99818023A00A4FD8BA531586BB8B1DC9AE080293B6972B7FA444285CC00AE492BC910C1697B5BDD8425409700562F471201186C0120004322B42489A200D4138A71AA796D00374978FE07B2314E99BFB6E909678A0"

bits = ""
ptr = 0
version_sum = 0

for h in hex_string:
  bits += h2b[h]

val, bits = process_packet(bits)
print("processed packet, remaining bits are:",bits)
print("version sum is",version_sum)
print("val is", val)

operator packet, length type 1
operator packet, length type 1
literal value is: 1161
operator packet, length type 1
operator packet, length type 0
literal value is: 2
literal value is: 4
literal value is: 13
operator packet, length type 1
literal value is: 6
literal value is: 6
literal value is: 3
operator packet, length type 1
literal value is: 5250146
operator packet, length type 1
literal value is: 246310804
literal value is: 246310804
operator packet, length type 0
operator packet, length type 1
literal value is: 4401420
literal value is: 55853
literal value is: 6
literal value is: 3759
operator packet, length type 1
operator packet, length type 1
operator packet, length type 1
operator packet, length type 1
operator packet, length type 0
operator packet, length type 1
operator packet, length type 0
operator packet, length type 1
operator packet, length type 0
operator packet, length type 0
operator packet, length type 1
operator packet, length type 0
operator packet, length type 0

In [188]:
target = [[20,30],[-10,-5]]

In [228]:
maxTri = 300
triNumbers =  [1]
for i in range(2,maxTri+1):
    triNumbers.append(i+triNumbers[-1])

In [229]:
print(triNumbers)

[1, 3, 6, 10, 15, 21, 28, 36, 45, 55, 66, 78, 91, 105, 120, 136, 153, 171, 190, 210, 231, 253, 276, 300, 325, 351, 378, 406, 435, 465, 496, 528, 561, 595, 630, 666, 703, 741, 780, 820, 861, 903, 946, 990, 1035, 1081, 1128, 1176, 1225, 1275, 1326, 1378, 1431, 1485, 1540, 1596, 1653, 1711, 1770, 1830, 1891, 1953, 2016, 2080, 2145, 2211, 2278, 2346, 2415, 2485, 2556, 2628, 2701, 2775, 2850, 2926, 3003, 3081, 3160, 3240, 3321, 3403, 3486, 3570, 3655, 3741, 3828, 3916, 4005, 4095, 4186, 4278, 4371, 4465, 4560, 4656, 4753, 4851, 4950, 5050, 5151, 5253, 5356, 5460, 5565, 5671, 5778, 5886, 5995, 6105, 6216, 6328, 6441, 6555, 6670, 6786, 6903, 7021, 7140, 7260, 7381, 7503, 7626, 7750, 7875, 8001, 8128, 8256, 8385, 8515, 8646, 8778, 8911, 9045, 9180, 9316, 9453, 9591, 9730, 9870, 10011, 10153, 10296, 10440, 10585, 10731, 10878, 11026, 11175, 11325, 11476, 11628, 11781, 11935, 12090, 12246, 12403, 12561, 12720, 12880, 13041, 13203, 13366, 13530, 13695, 13861, 14028, 14196, 14365, 14535, 14706, 14

In [192]:
xCandidates = filter(lambda x: x in range(target[0][0],target[0][1]+1),triNumbers)

In [194]:
print(list(xCandidates))

[21, 28]


In [245]:
def simulate(vx0,vy0,target):
    xmin,xmax,ymin,ymax = target
    maxHeight = 0
    x,y = 0,0
    vx = vx0
    vy = vy0
    success = False
    while True:
        x+=vx
        y+=vy
#         print(x,y)
        vy -= 1
        if vx>0:
            vx-=1
        elif vx<0:
            vx +=1
        maxHeight = max(maxHeight,y)
        if x in range(xmin,xmax+1) and y in range(ymin,ymax+1):
            success = True
            break
        if y<ymin:
            break
            
    return success,maxHeight


In [196]:
target = [20,30,-10,-5]

In [202]:
print(simulate(7,5,target))

7 5
13 9
18 12
22 14
25 15
27 15
28 14
28 12
28 9
28 5
28 0
28 -6
(True, 15)


In [203]:
t2 = [257,286,-101,-57]

In [249]:
def p2():
    vxmin=22
    vxmax=287
    vymin=-101
    vymax=101
#     vxmin = 6
#     vxmax = 30
#     vymin = -20
#     vymax = 30
#     target = [20,30,-10,-5]
    target = [257,286,-101,-57]
    maxvx,maxvy,maxheight = 0,0,0
    cnt = 0
    for vx in range(vxmin,vxmax+1):
        for vy in range(vymin,vymax+1):
            s,h = simulate(vx,vy,target)
            if s:
                cnt += 1
            if s and h> maxheight:
                maxheight = h
                maxvx = vx
                maxvy = vy
    print(maxvx,maxvy,maxheight,cnt)

In [250]:
p2()

23 100 5050 2223


In [239]:
max([x if x in range(10,22) else -1 for x in triNumbers])
    

21

In [242]:
def part1(target):
    xmin,xmax,ymin,ymax = target
    # create triNumbers
    maxTri = 300
    triNumbers =  [1]
    for i in range(2,maxTri+1):
        triNumbers.append(i+triNumbers[-1])
        
    # check if "simple" solution works, otherwise surrender ;)
    if any(x in triNumbers for x in range(xmin,xmax+2)):
        return triNumbers[max(abs(ymin),abs(ymax))-2]
    else:
        return -1

In [244]:
print(p1([257,286,-101,-57]))

5050


In [ ]:
def part2(target):
    xmin,xmax,ymin,ymax = target
    # create triNumbers
    maxTri = 300
    triNumbers =  [1]
    for i in range(2,maxTri+1):
        triNumbers.append(i+triNumbers[-1])
    vxmax = xmax +1
    # find vx_min
    vxmin = 0
    for i,v in enumerate(triNumbers):
        if v >= xmin:
            vxmin = i+1
    print(vxmin,vxmax)